# Lab 04 : VGG architecture - exercise

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from random import randint
import utils
import time

### With or without GPU?

It is recommended to run this code on GPU:<br> 
* Time for 1 epoch on CPU : 841 sec (14.02 min)<br> 
* Time for 1 epoch on GPU : 9 sec w/ GeForce GTX 1080 Ti <br>

In [ ]:
device= torch.device("cuda")
device= torch.device("cpu")
print(device)

### Download the CIFAR dataset 

In [ ]:
from utils import check_cifar_dataset_exists
data_path=check_cifar_dataset_exists()

train_data=torch.load(data_path+'cifar/train_data.pt')
train_label=torch.load(data_path+'cifar/train_label.pt')
test_data=torch.load(data_path+'cifar/test_data.pt')
test_label=torch.load(data_path+'cifar/test_label.pt')

print(train_data.size())
print(test_data.size())

### Compute mean pixel intensity over all training set and all channels

In [ ]:
mean= train_data.mean()

print(mean)

### Compute standard deviation

In [ ]:
std= train_data.std()

print(std)

### Make a VGG convnet class. 

In [ ]:
class VGG_convnet(nn.Module):

    def __init__(self):

        super(VGG_convnet, self).__init__()

        # block 1:         3 x 32 x 32 --> 64 x 16 x 16        
        self.conv1a = nn.Conv2d(3,   64,  kernel_size=3, padding=1 )
        self.conv1b = nn.Conv2d(64,  64,  kernel_size=3, padding=1 )
        self.pool1  = nn.MaxPool2d(2,2)

        # block 2:         64 x 16 x 16 --> 128 x 8 x 8
        self.conv2a = # COMPLETE HERE
        self.conv2b = # COMPLETE HERE
        self.pool2  = # COMPLETE HERE

        # block 3:         128 x 8 x 8 --> 256 x 4 x 4        
        self.conv3a = # COMPLETE HERE
        self.conv3b = # COMPLETE HERE
        self.pool3  = # COMPLETE HERE
        
        #block 4:          256 x 4 x 4 --> 512 x 2 x 2
        self.conv4a = # COMPLETE HERE
        self.pool4  = # COMPLETE HERE

        # linear layers:   512 x 2 x 2 --> 2048 --> 4096 --> 4096 --> 10
        self.linear1 = # COMPLETE HERE
        self.linear2 = # COMPLETE HERE
        self.linear3 = # COMPLETE HERE


    def forward(self, x):

        # block 1:         3 x 32 x 32 --> 64 x 16 x 16
        x = self.conv1a(x)
        x = F.relu(x)
        x = self.conv1b(x)
        x = F.relu(x)
        x = self.pool1(x)

        # block 2:         64 x 16 x 16 --> 128 x 8 x 8
        x = # COMPLETE HERE
        x = # COMPLETE HERE
        x = # COMPLETE HERE
        x = # COMPLETE HERE
        x = # COMPLETE HERE

        # block 3:         128 x 8 x 8 --> 256 x 4 x 4
        x = # COMPLETE HERE
        x = # COMPLETE HERE
        x = # COMPLETE HERE
        x = # COMPLETE HERE
        x = # COMPLETE HERE

        #block 4:          256 x 4 x 4 --> 512 x 2 x 2
        x = # COMPLETE HERE
        x = # COMPLETE HERE
        x = # COMPLETE HERE

        # linear layers:   512 x 2 x 2 --> 2048 --> 4096 --> 4096 --> 10
        x = x.view(# COMPLETE HERE)
        x = # COMPLETE HERE
        x = # COMPLETE HERE
        x = # COMPLETE HERE
        x = # COMPLETE HERE
        x = # COMPLETE HERE 
        
        return x

### Build the net. How many parameters in total? (the three layer net had 2 million parameters)

In [ ]:
net = # COMPLETE HERE

print(net)
utils.display_num_param(net)

### Send the weights of the networks to the GPU (as well as the mean and std)

In [ ]:
net = # COMPLETE HERE

mean = mean.to(device)

std = std.to(device)

### Choose the criterion, batch size, and initial learning rate. Select the following:
* batch size =128
* initial learning rate =0.25


In [ ]:
criterion = nn.CrossEntropyLoss()
my_lr= # COMPLETE HERE
bs=    # COMPLETE HERE

### Function to evaluate the network on the test set

In [ ]:
def eval_on_test_set():

    running_error=0
    num_batches=0

    for i in range(0,10000,bs):

        minibatch_data =  test_data[i:i+bs]
        minibatch_label= test_label[i:i+bs]

        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
        
        inputs = (minibatch_data - mean)/std

        scores=net( inputs ) 

        error = utils.get_error( scores , minibatch_label)

        running_error += error.item()

        num_batches+=1

    total_error = running_error/num_batches
    print( 'error rate on test set =', total_error*100 ,'percent')

### Do 20 passes through the training set. Divide the learning rate by 2 at epoch 10, 14 and 18.

In [ ]:
start=time.time()

for epoch in range(1,20):
    
    # divide the learning rate by 2 at epoch 10, 14 and 18
    if # COMPLETE HERE
        my_lr = # COMPLETE HERE
    
    # create a new optimizer at the beginning of each epoch: give the current learning rate.   
    optimizer=torch.optim.SGD( net.parameters() , lr=my_lr )
        
    # set the running quatities to zero at the beginning of the epoch
    running_loss=0
    running_error=0
    num_batches=0
    
    # set the order in which to visit the image from the training set
    shuffled_indices=torch.randperm(50000)
 
    for count in range(0,50000,bs):
    
        # Set the gradients to zeros
        optimizer.zero_grad()
        
        # create a minibatch       
        indices=shuffled_indices[count:count+bs]
        minibatch_data =  train_data[indices]
        minibatch_label=  train_label[indices]
        
        # send them to the gpu
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
        
        # normalize the minibatch (this is the only difference compared to before!)
        inputs = (minibatch_data - mean)/std
        
        # tell Pytorch to start tracking all operations that will be done on "inputs"
        inputs.requires_grad_()

        # forward the minibatch through the net 
        scores=net( inputs ) 

        # Compute the average of the losses of the data points in the minibatch
        loss =  criterion( scores , minibatch_label) 
        
        # backward pass to compute dL/dU, dL/dV and dL/dW   
        loss.backward()

        # do one step of stochastic gradient descent: U=U-lr(dL/dU), V=V-lr(dL/dU), ...
        optimizer.step()
        

        # START COMPUTING STATS
        
        # add the loss of this batch to the running loss
        running_loss += loss.detach().item()
        
        # compute the error made on this batch and add it to the running error       
        error = utils.get_error( scores.detach() , minibatch_label)
        running_error += error.item()
        
        num_batches+=1        
    
    
    # compute stats for the full training set
    total_loss = running_loss/num_batches
    total_error = running_error/num_batches
    elapsed = (time.time()-start)/60
    

    print('epoch=',epoch, '\t time=', elapsed,'min','\t lr=', my_lr  ,'\t loss=', total_loss , '\t error=', total_error*100 ,'percent')
    eval_on_test_set() 
    print(' ')
    
           

### Choose image at random from the test set and see how good/bad are the predictions

In [ ]:
# choose a picture at random
idx=randint(0, 10000-1)
im=test_data[idx]

# diplay the picture
utils.show(im)

# send to device, rescale, and view as a batch of 1 
im = im.to(device)
im= (im-mean) / std
im=im.view(1,3,32,32)

# feed it to the net and display the confidence scores
scores =  net(im) 
probs= F.softmax(scores, dim=1)
utils.show_prob_cifar(probs.cpu())